# Spacy.io NLP stuff

[Spacy](https://spacy.io/) is "Industrial-Strength Natural Language Processing" (NLP)

```bash
pip install spacy
python -m spacy download en # downloads English NLP model info
```

There are other, non-English [language models](https://spacy.io/usage/models).

Let's load the Tesla IPO again:

In [17]:
! curl https://www.sec.gov/Archives/edgar/data/1318605/000119312510017054/ds1.htm > /tmp/TeslaIPO.html

In [18]:
import sys
from bs4 import BeautifulSoup

def html2text(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text()
    return text

with open("/tmp/TeslaIPO.html", "r") as f:
    html_text = f.read()
tsla = html2text(html_text)
print(tsla[0:100].split())

['S-1', '1', 'ds1.htm', 'REGISTRATION', 'STATEMENT', 'ON', 'FORM', 'S-1', 'Registration', 'Statement', 'on', 'Form', 'S-1', 'Table', 'of', 'Co']


## Tokenizing with Spacy

In [20]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [21]:
doc = nlp(tsla[0:5000])
type(doc)

spacy.tokens.doc.Doc

In [10]:
for token in doc[:30]:
    if len(str(token).strip())>0:
        print(token.text.strip())

S-1
1
ds1.htm
REGISTRATION
STATEMENT
ON
FORM
S-1
Registration
Statement
on
Form
S-1
Table
of
Contents
As
filed
with
the
Securities
and
Exchange


## Parts of speech

In [11]:
import pandas as pd
winfo = []
for token in doc[100:120]:
    winfo.append([token.text, token.pos_, token.is_stop])
    
pd.DataFrame(data=winfo, columns=['word','part of speech', 'stop word'])

word part of speech  stop word
0     jurisdiction           NOUN      False
1               of            ADP       True
2    incorporation           NOUN      False
3               or          CCONJ       True
4     organization           NOUN      False
5                )          PUNCT      False
6           \n \n           SPACE      False
7                (          PUNCT      False
8          Primary          PROPN      False
9         Standard          PROPN      False
10      Industrial          PROPN      False
11  Classification          PROPN      False
12            Code          PROPN      False
13          Number          PROPN      False
14               )          PUNCT      False
15          \n \n           SPACE      False
16               (          PUNCT      False
17          I.R.S.          PROPN      False
18        Employer          PROPN      False
19  Identification          PROPN      False

In [14]:
winfo = []
for ent in doc.ents[:20]:
    winfo.append([ent.text, ent.label_])
pd.DataFrame(data=winfo, columns=['word', 'label'])

word     label
0                                        1  CARDINAL
1   the Securities and Exchange Commission       ORG
2                         January 29, 2010      DATE
3                          UNITED STATES         ORG
4                           SECURITIES AND       ORG
5                                 EXCHANGE       ORG
6                               Washington       GPE
7                                     D.C.       GPE
8                                     FORM       ORG
9                              STATEMENT      PERSON
10                            Tesla Motors       ORG
11                                   Exact       ORG
12                              Registrant       ORG
13                              91-2197729  CARDINAL
14                                   State       ORG
15   I.R.S. Employer Identification Number       ORG
16                         Deer Creek Road       FAC
17                               Palo Alto       GPE
18                              California       GPE
19                                     650  CARDINAL

**Word vectors**

In [16]:
winfo = []
for t in doc[100:110]:
    winfo.append([t.text, t.vector])
pd.DataFrame(data=winfo, columns=['word', 'vector'])

word                                             vector
0   jurisdiction  [3.4729223, -1.4104788, -1.2004784, -0.5751643...
1             of  [-0.5583954, -1.4372051, -5.764125, -3.389831,...
2  incorporation  [6.166033, -1.3807518, -0.19106722, -1.9463961...
3             or  [2.0970569, 1.9565225, -4.5255666, -0.60888207...
4   organization  [7.056058, 0.38515526, 1.7288362, 0.21033919, ...
5              )  [2.3581414, 1.1409386, 1.0774156, -0.045503706...
6         \n \n   [0.3358876, -0.98331153, -4.240477, 0.67844033...
7              (  [2.8860693, 0.09956763, -2.7654376, 2.4729605,...
8        Primary  [-1.6166165, -3.735238, -2.8839834, -2.1430888...
9       Standard  [0.5131076, -4.1868796, -3.940477, -2.1611662,...

## Visualizing entities in notebook

In [39]:
from spacy import displacy
displacy.render(doc[100:180], style='ent')

## Splitting into sentences

In [24]:
winfo = []
for s in doc.sents:
    winfo.append([s.text])
pd.DataFrame(data=winfo, columns=['sentence'])

sentence
0   \nS-1\n1\nds1.htm\nREGISTRATION STATEMENT ON F...
1   Registration Statement on Form S-1\n\n\nTable ...
2   As filed with the Securities and Exchange Comm...
3         333-                \n      UNITED STATES  
4   SECURITIES AND EXCHANGE COMMISSION  Washington...
5                                                FORM
6                     S-1 \n REGISTRATION STATEMENT  
7                                             UNDER  
8   THE SECURITIES ACT OF 1933      Tesla Motors, ...
9   (Exact name of Registrant as\nspecified in its...
10                             Delaware\n \n3711\n \n
11  91-2197729\n\n (State or other jurisdiction of...
12  (I.R.S. Employer Identification Number) 3500 D...
13                                   (650) 413-4000  
14  (Address, including zip code, and telephone nu...
15      Palo Alto, California 94304  (650) 413-4000  
16  (Name, address, including zip code, and teleph...
17  Copies to:   \n\n\n\n\n\n\n Larry W. Sonsini D...
18                              (650) 493-9300\n  \n 
19  Kevin P. Kennedy Simpson Thacher & Bartlett LL...
20  Approximate date\nof commencement of proposed ...
21  If any of the\nsecurities being registered on ...
22                                                ¨  
23  If this Form is filed to register additional s...
24                                                ¨  
25  If this Form is a post-effective amendment fil...
26                                                ¨  
27  If this Form is a post-effective amendment fil...
28                                                ¨  
29  Indicate by check mark whether the registrant ...
30  See the definitions of “large accelerated file...
31  (Check one):   \n\n\n\n\n\n\n\n\n\n\nLarge acc...
32  (Do not check if a smaller reporting company)\...
33               Smaller reporting company\n \n¨     
34  CALCULATION OF\nREGISTRATION FEE   \n\n\n\n\n\...
35  Title of Each Class of Securities to be Regist...
36                                  Price(1)(2)\n \n 
37  Amount ofRegistration Fee\n\n Common Stock, $0...
38  (1)\nEstimated solely for the purpose of compu...
39  (2)\nIncludes shares the underwriters have the...
40  The Registrant hereby amends this Registration...
41                                Table of Contents\n
42  The information in this preliminary prospectus...
43  These securities\nmay not be sold until the re...
44  This preliminary prospectus is not an offer to...
45  SUBJECT TO COMPLETION, DATED JANUARY 29, 2010 ...
46                                 Common Stock      
47  This is an initial public offering of shares o...
48  Tesla Motors is offering             of the sh...
49  The selling stockholders identified in this pr...
50  Tesla Motors will not receive any of the proce...
51  Prior to this offering, there has been no publ...
52  It is\ncurrently estimated that the initial pu...

## Exercise

Extract any word in the TSLA doc that is a number per Spacy. See [Spacy 101](https://spacy.io/usage/spacy-101). Your output should look like (assuming you used `doc = nlp(tsla[0:5000])`):

```
[1, 29, 2010, 20549, 1933, 3711, 91, 2197729, 3500, 94304, 650, 413, 4000, 3500, 94304, 650, 413, 4000, 650, 94304, 650, 493, 9300, 2550, 94304, 650, 251, 5000, 415, one, 0.001, 100,000,000, 7,130, 1, 457, 1933, 2, 1933, 29, 2010]
```

See [solution](https://github.com/parrt/msds692/tree/master/notes/code/spacy) if you get stuck.